In [1]:
import torch
from torch import nn
import re
import random
import tqdm
import time

In [5]:
!pip install wget

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9680 sha256=4d733d0a9abc1b712f66c3a09f6ba3c143d67a36738f58fa3629c49b502f3215
  Stored in directory: c:\users\анатолий\appdata\local\pip\cache\wheels\40\b3\0f\a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [8]:
!wget https://s3.amazonaws.com/text-datasets/nietzsche.txt

--2023-09-21 22:14:49--  https://s3.amazonaws.com/text-datasets/nietzsche.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.16.46, 52.217.37.86, 52.217.74.86, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.16.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 600901 (587K) [text/plain]
Saving to: 'nietzsche.txt'

     0K .......... .......... .......... .......... ..........  8%  330K 2s
    50K .......... .......... .......... .......... .......... 17%  337K 1s
   100K .......... .......... .......... .......... .......... 25%  559K 1s
   150K .......... .......... .......... .......... .......... 34%  830K 1s
   200K .......... .......... .......... .......... .......... 42% 10,3M 1s
   250K .......... .......... .......... .......... .......... 51%  351K 1s
   300K .......... .......... .......... .......... .......... 59% 14,8M 0s
   350K .......... .......... .......... .......... .......... 68% 10,3M 0s
   400K .......... ........

In [9]:
with open('nietzsche.txt', encoding='utf-8') as f:
    text = f.read().lower()
print('length:', len(text))
text = re.sub('[^a-z ]', ' ', text)
text = re.sub('\s+', ' ', text)

length: 600893


In [10]:
text[:100]

'preface supposing that truth is a woman what then is there not ground for suspecting that all philos'

In [11]:
INDEX_TO_CHAR = sorted(list(set(text)))
CHAR_TO_INDEX = {c: i for i, c in enumerate(INDEX_TO_CHAR)}

In [12]:
CHAR_TO_INDEX

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [13]:
MAX_LEN = 40
STEP = 3
SENTENCES = []
NEXT_CHARS = []
for i in range(0, len(text) - MAX_LEN, STEP):
    SENTENCES.append(text[i: i + MAX_LEN])
    NEXT_CHARS.append(text[i + MAX_LEN])
print('Num sents:', len(SENTENCES))

Num sents: 193075


In [14]:
print('Vectorization...')
X = torch.zeros((len(SENTENCES), MAX_LEN), dtype=int)
Y = torch.zeros((len(SENTENCES)), dtype=int)
for i, sentence in enumerate(SENTENCES):
    for t, char in enumerate(sentence):
        X[i, t] = CHAR_TO_INDEX[char]
    Y[i] = CHAR_TO_INDEX[NEXT_CHARS[i]]

Vectorization...


In [15]:
X[0:1], Y[0]

(tensor([[16, 18,  5,  6,  1,  3,  5,  0, 19, 21, 16, 16, 15, 19,  9, 14,  7,  0,
          20,  8,  1, 20,  0, 20, 18, 21, 20,  8,  0,  9, 19,  0,  1,  0, 23, 15,
          13,  1, 14,  0]]),
 tensor(23))

In [16]:
BATCH_SIZE=512
dataset = torch.utils.data.TensorDataset(X, Y)
data = torch.utils.data.DataLoader(dataset, BATCH_SIZE, shuffle=True)


In [17]:
class NeuralNetwork(nn.Module):
    def __init__(self, rnnClass, dictionary_size, embedding_size, num_hiddens, num_classes):
        super().__init__()
        
        self.num_hiddens = num_hiddens
        self.embedding = nn.Embedding(dictionary_size, embedding_size)
        self.hidden = rnnClass(embedding_size, num_hiddens, batch_first=True)
        self.output = nn.Linear(num_hiddens, num_classes)
        
    def forward(self, X):
        out = self.embedding(X)
        _, state = self.hidden(out)
        predictions = self.output(state[0].squeeze())
        return predictions

In [18]:
model = NeuralNetwork(nn.GRU, len(CHAR_TO_INDEX), 64, 128, len(CHAR_TO_INDEX))

In [19]:
X.shape

torch.Size([193075, 40])

In [20]:
model(X[0:1])

tensor([-0.0791, -0.2085, -0.0455, -0.1693, -0.2564,  0.0378, -0.1432, -0.1375,
        -0.0876, -0.0774, -0.0693, -0.1199,  0.1950,  0.0108, -0.1786,  0.0804,
         0.0004,  0.1309, -0.0236,  0.0134,  0.0097,  0.1423, -0.1204, -0.0674,
        -0.0557,  0.1333,  0.0175], grad_fn=<ViewBackward0>)

In [21]:
embedding = nn.Embedding(len(INDEX_TO_CHAR), 15)
rnn = nn.LSTM(15,128, batch_first=True)

In [22]:
o, s = rnn(embedding(X[0:10]))
o.shape, len(s), s[0].shape, s[1].shape

(torch.Size([10, 40, 128]),
 2,
 torch.Size([1, 10, 128]),
 torch.Size([1, 10, 128]))

In [30]:
rnn = nn.GRU(15,128, batch_first=True)
o, s = rnn(embedding(X[0:10]))
o.shape, len(s), s[0].shape

(torch.Size([10, 40, 128]), 1, torch.Size([10, 128]))

In [31]:
o, s = rnn(embedding(X[0:10]))

In [38]:
o.shape, s[0].shape, s[1].shape

IndexError: index 1 is out of bounds for dimension 0 with size 1

In [33]:
model = model.cuda()

In [34]:
def sample(preds):
    softmaxed = torch.softmax(preds, 0)
    probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()
    return probas.argmax()

def generate_text():
    start_index = random.randint(0, len(text) - MAX_LEN - 1)

    generated = ''
    sentence = text[start_index: start_index + MAX_LEN]
    generated += sentence

    for i in range(MAX_LEN):
        x_pred = torch.zeros((1, MAX_LEN), dtype=int)
        for t, char in enumerate(generated[-MAX_LEN:]):
            x_pred[0, t] = CHAR_TO_INDEX[char]

        preds = model(x_pred.cuda())[0].cpu()
        next_char = INDEX_TO_CHAR[sample(preds)]
        generated = generated + next_char

    print(generated[:MAX_LEN] + '|' + generated[MAX_LEN:])

In [35]:
generate_text()

ValueError: `probs` parameter must be at least one-dimensional.

In [36]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [37]:
for ep in range(100):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    model.train()
    for X_b, y_b in data:
        X_b, y_b = X_b.cuda(), y_b.cuda()
        optimizer.zero_grad()
        answers = model(X_b)
        loss = criterion(answers, y_b)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    model.eval()
    generate_text()

Epoch 0. Time: 6.682, Train loss: 2.106


ValueError: `probs` parameter must be at least one-dimensional.